In [70]:
import pandas as pd
import time
import re
import numpy as np

In [2]:
#Test
df = pd.DataFrame(data={'Item':['A','B','C','D'], 
                        'StartDate':pd.to_datetime(['2018-01-01','2018-01-07', '2017-12-30','2018-01-03']),
                        'EndDate': pd.to_datetime(['2018-01-04','2018-01-08','2018-01-05','2018-01-05']),
                        'RandomStuff':['sdfe','adfe','dafsdfew','fjt'],
                        'CT':[12,14,21,12]},
                 )

df['dayDiff'] = df.apply(lambda avg: avg['CT']/((avg['EndDate']-avg['StartDate']).days +1),axis=1)
#print(df)


In [3]:
start = time.time()

new_df = df.apply(lambda x: pd.Series(
        {
        'avg':x['CT']/((x['EndDate']-x['StartDate']).days +1),
        'item':x['Item'],
        'date_range':pd.date_range(x['StartDate'],x['EndDate'])
        }), 
    axis=1).set_index(['item','avg'])['date_range'].apply(pd.Series).stack().reset_index().drop('level_2',axis=1)

print(time.time()-start)
new_df

0.015245199203491211


,item,avg,0
0,A,3.0,2018-01-01
1,A,3.0,2018-01-02
2,A,3.0,2018-01-03
3,A,3.0,2018-01-04
4,B,7.0,2018-01-07
5,B,7.0,2018-01-08
6,C,3.0,2017-12-30
7,C,3.0,2017-12-31
8,C,3.0,2018-01-01
9,C,3.0,2018-01-02


In [4]:
all_item = list(set(df['Item']))

In [5]:
start = time.time()

# df = df.copy()
index = pd.MultiIndex.from_product([all_item,pd.date_range(start = df.StartDate.min(), end = df.EndDate.max())], names=['Item', 'date'])
ori_df = pd.DataFrame(index = index, data = {'counts':0})

dates_list = ori_df.index.get_level_values(1)
items_list = ori_df.index.get_level_values(0)

for row in df.itertuples():
    ori_df.counts[(items_list == row.Item) & (dates_list>=row.StartDate) & (dates_list<= row.EndDate)] += row[6]

# to remove 0 if needed
ori_df = ori_df[(ori_df.counts != 0)]

ori_df.reset_index(inplace=True)

print(time.time()-start)
ori_df

0.031501054763793945


,Item,date,counts
0,A,2018-01-01,3
1,A,2018-01-02,3
2,A,2018-01-03,3
3,A,2018-01-04,3
4,D,2018-01-03,4
5,D,2018-01-04,4
6,D,2018-01-05,4
7,B,2018-01-07,7
8,B,2018-01-08,7
9,C,2017-12-30,3


In [6]:
ori_df = pd.merge(ori_df, df, how='inner', on='Item')
ori_df

,Item,date,counts,StartDate,EndDate,RandomStuff,CT,dayDiff
0,A,2018-01-01,3,2018-01-01,2018-01-04,sdfe,12,3.0
1,A,2018-01-02,3,2018-01-01,2018-01-04,sdfe,12,3.0
2,A,2018-01-03,3,2018-01-01,2018-01-04,sdfe,12,3.0
3,A,2018-01-04,3,2018-01-01,2018-01-04,sdfe,12,3.0
4,D,2018-01-03,4,2018-01-03,2018-01-05,fjt,12,4.0
5,D,2018-01-04,4,2018-01-03,2018-01-05,fjt,12,4.0
6,D,2018-01-05,4,2018-01-03,2018-01-05,fjt,12,4.0
7,B,2018-01-07,7,2018-01-07,2018-01-08,adfe,14,7.0
8,B,2018-01-08,7,2018-01-07,2018-01-08,adfe,14,7.0
9,C,2017-12-30,3,2017-12-30,2018-01-05,dafsdfew,21,3.0


In [123]:
#Test
def sprint_parse(text):
    sprint = re.search('sprint \d*\.?\d*', text.lower())
    if sprint is None:
        return 0
    else:
        #return re.sub('[a-z]', '', sprint.group(0))
        return pd.to_numeric(re.sub("[a-z]", "", sprint.group(0)), errors='coerce')

ndf = pd.DataFrame(data={'Item':['A','B','C','D'], 
                        'StartDate':pd.to_datetime(['2018-01-01','2018-01-07', '2017-12-30','2018-01-03']),
                        'EndDate': pd.to_datetime(['2018-01-04','2018-01-08','2018-01-05','2018-01-05']),
                        'RandomStuff':['12 sprint 1','sprint 12 13.4','R3 sprint 13.3 t', 'xxxxx'],
                        'CT':[12,14,21,12]},
                 )

ndf.RandomStuff = ndf['RandomStuff'].apply(sprint_parse)
ndf.rename(columns={'Item': '0'}, index=str)
ndf


,Item,StartDate,EndDate,RandomStuff,CT
0,A,2018-01-01,2018-01-04,1.0,12
1,B,2018-01-07,2018-01-08,12.0,14
2,C,2017-12-30,2018-01-05,13.3,21
3,D,2018-01-03,2018-01-05,0.0,12


'aaa'